In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
game_reviews_df = pd.read_csv('steam-200k.csv')
#print(game_reviews_df)

clean_game_reviews_df = game_reviews_df.drop(['0', 'purchase'], axis=1)
clean_game_reviews_df = clean_game_reviews_df.rename({'151603712': 'User', 'The Elder Scrolls V Skyrim': 'Game', '1.0': 'PlayTime'}, axis=1)
matrix = clean_game_reviews_df.pivot_table(columns='Game', index='User', values='PlayTime', fill_value=0)
matrix = matrix.dropna(thresh=5, axis=0)

In [4]:
def center(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row

matrix_std = matrix.apply(center)
print(matrix_std)

Game       007 Legends  0RBITALIS  \
User                                
5250         -0.000081  -0.000212   
76767        -0.000081  -0.000212   
86540        -0.000081  -0.000212   
103360       -0.000081  -0.000212   
144736       -0.000081  -0.000212   
...                ...        ...   
309554670    -0.000081  -0.000212   
309626088    -0.000081  -0.000212   
309812026    -0.000081  -0.000212   
309824202    -0.000081  -0.000212   
309903146    -0.000081  -0.000212   

Game       1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)  \
User                                                                    
5250                                               -0.000192            
76767                                              -0.000192            
86540                                              -0.000192            
103360                                             -0.000192            
144736                                             -0.000192            
...    

In [5]:
def gameRec(game):
    centered_matrix_values = matrix_std[game]
    #Calcula la correlación de pearson de un juego con los demás
    centered_matrix_values = matrix.corrwith(centered_matrix_values).dropna()
    #crea un dataframe que muestra las veces que cada juego ha sido jugado y la media del tiempo que ha sido jugado
    gameData = clean_game_reviews_df.groupby('Game').agg({'PlayTime': [np.size, np.mean]})
    #filtro que elimina los juegos jugados por menos de 100 usuarios.
    gameSim = gameData['PlayTime']['size'] >= 100
    print(gameSim)
    df = gameData[gameSim].join(pd.DataFrame(centered_matrix_values, columns=['similarity']))
    return df.sort_values(['similarity'], ascending=False)[:6]

In [6]:
gameRec("Fallout 4")

Game
007 Legends                                                   False
0RBITALIS                                                     False
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)    False
10 Second Ninja                                               False
10,000,000                                                    False
                                                              ...  
sZone-Online                                                   True
samurai_jazz                                                  False
the static speaks my name                                     False
theHunter                                                      True
theHunter Primal                                              False
Name: size, Length: 5155, dtype: bool


/tmp/ipykernel_409/729552535.py:10: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df = gameData[gameSim].join(pd.DataFrame(centered_matrix_values, columns=['similarity']))


,"(PlayTime, size)","(PlayTime, mean)",similarity
Game,,,
Fallout 4,335,32.629254,1.000000
Alan Wake,218,5.389908,0.345539
Darksiders,245,5.506122,0.315038
Middle-earth Shadow of Mordor,209,11.657895,0.269952
Company of Heroes,241,3.274689,0.269796
Borderlands,329,14.972036,0.263348
